In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
!nvidia-smi

Sat Mar  5 16:46:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import tensorflow as tf

tf.__version__

'1.15.2'

In [ ]:
import os
if os.path.isdir("/content/drive/My Drive/StyleGan2/stylegan2-ada"):
    %cd "/content/drive/My Drive/StyleGan2/stylegan2-ada"
else:
    #install script
    %cd "/content/drive/My Drive/"
    #!mkdir StyleGan2
    %cd StyleGan2
    !git clone https://github.com/NVlabs/stylegan2-ada.git
    %cd stylegan2-ada
    !mkdir downloads
    !mkdir datasets

/content/drive/My Drive/StyleGan2/stylegan2-ada


In [ ]:
#update this to the path to your image folder
dataset_path = "/content/drive/MyDrive/StyleGan2/data"
#give your dataset a name
dataset_name = "data"

#you don't need to edit anything here
!python dataset_tool.py create_from_images ./datasets/{dataset_name} {dataset_path}

Loading images from "/content/drive/MyDrive/StyleGan2/data"
Creating dataset "./datasets/data"
dataset_tool.py:96: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
Added 553 images.


In [ ]:
#this name must EXACTLY match the dataset name you used when creating the .tfrecords file
dataset_name = "data"
#how often should the model generate samples and a .pkl file
snapshot_count = 10
#should the images be mirrored left to right?
mirrored = True
#should the images be mirrored top to bottom?
mirroredY = False
#metrics? 
metric_list = None
#augments
augs = "bg"

#
# this is the most important cell to update
#
# running it for the first time? set it to ffhq(+resolution)
# resuming? get the path to your latest .pkl file and use that
#resume_from = "ffhq1024"
resume_from = "/content/drive/MyDrive/StyleGan2/stylegan2-ada/results/00002-data-mirror-auto1-bg-resumecustom/network-snapshot-000032.pkl" 

In [ ]:
!python train.py --outdir ./results --snap={snapshot_count} --data=./datasets/{dataset_name} --augpipe={augs} --mirror={mirrored} --metrics={metric_list} --resume={resume_from} --augpipe="bg"

In [ ]:
!python generate.py --outdir ./gen --trunc=0.5 --seeds=201-400 --network={resume_from}

In [ ]:
# trunc determines the diversity of the images higher trunc more divers but lower res

In [ ]:
!python calc_metrics.py --metrics=fid50k_full \
    --network={resume_from}

Loading network from "/content/drive/MyDrive/StyleGan2/stylegan2-ada/results/00002-data-mirror-auto1-bg-resumecustom/network-snapshot-000032.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Compiling... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Compiling... Loading... Done.

Gs                            Params    OutputShape         WeightShape     
---                           ---       ---                 ---             
latents_in                    -         (?, 512)            -               
labels_in                     -         (?, 0)              -               
G_mapping/Normalize           -         (?, 512)            -               
G_mapping/Dense0              262656    (?, 512)            (512, 512)      
G_mapping/Dense1              262656    (?, 512)            (512, 512)      
G_mapping/Broadcast           -         (?, 16, 512)        -               
dlatent_avg                   -         (512,)              -               
T